In [123]:
# import dependencies
import h5py
import numpy as np
import gzip
import os

In [124]:
HDF5_PATH = "../density_test/634500.hdf5"
MIN_Z = 32
MAX_Z = 48
MIN_XY = 0
MAX_XY = 80
OUTER_CONSTRICTION_RADIUS = 18.5 #nm
INNER_CONSTRICTION_RADIUS = 11 #nm
N_BEADS = 512
SLAB_WIDTH = 15 #nm
FLOATER_RADIUS = 30 #A

In [125]:
SINGLE_BEAD_MASS = 2.2 #kd
# SINGLE_BEAD_MASS = 3.6531884400000005e-21 #g
FLOATER_DENSITY = 27/8000 # kd/A^3
half_torus_volume = (np.pi ** 2) * OUTER_CONSTRICTION_RADIUS * ((SLAB_WIDTH / 2) ** 2) #nm^3
hourglass_volume = np.pi * (OUTER_CONSTRICTION_RADIUS ** 2) * SLAB_WIDTH - half_torus_volume #nm^3

In [126]:
def get_fg_counts_maps(path, min_z=MIN_Z, max_z=MAX_Z):
    with open(path, "rb") as f:
        f = h5py.File(f, "r")
        fg_data = f["fg_xyz_hist"]
        n_counts_maps = int(len(fg_data.keys()) / 2)
        fg_counts_maps = np.zeros(shape=(MAX_XY - MIN_XY, MAX_XY - MIN_XY, max_z - min_z, n_counts_maps))
        for i, key in enumerate(fg_data.keys()):
            if i % 2 == 0:
                fg_counts_maps[:, :, :, int(i / 2)] += np.array(fg_data[key][MIN_XY:MAX_XY,MIN_XY:MAX_XY,min_z:max_z])
            else:
                fg_counts_maps[:, :, :, int((i - 1) / 2)] += np.array(fg_data[key][MIN_XY:MAX_XY,MIN_XY:MAX_XY,min_z:max_z])
        return fg_counts_maps
                
                
def get_floater_counts_maps(path, min_z=MIN_Z, max_z=MAX_Z):
    with open(path, "rb") as f:
        f = h5py.File(f, "r")
        floater_data = f["floater_xyz_hist"]
        n_counts_maps = len(floater_data.keys())
        floater_counts_maps = np.zeros(shape=(MAX_XY - MIN_XY, MAX_XY - MIN_XY, max_z - min_z, n_counts_maps))
        floater_sizes = []
        for i, key in enumerate(floater_data.keys()):
            floater_counts_maps[:, :, :, i] = np.array(floater_data[key][MIN_XY:MAX_XY,MIN_XY:MAX_XY,min_z:max_z])
            # Get the size of the floater in angstrom according to its IMP type name.
            size = int(float(''.join(map(str, list(filter(str.isdigit, key))))))
            floater_sizes.append(size)
        return floater_counts_maps, floater_sizes
    
def calc_normalization_factor(path):
    fg_counts_maps = get_fg_counts_maps(path, 0, 80)
    return np.sum(fg_counts_maps) / N_BEADS

def calculate_fg_density(counts_maps, norm_factor):
    # print(norm_factor)
    beads_in_channel = np.sum(counts_maps) / norm_factor
    total_mass = beads_in_channel * SINGLE_BEAD_MASS
    density = total_mass / hourglass_volume
    return density # kd / nm^3

def calculate_floater_density(counts_maps, norm_factor):
    # print(norm_factor)
    floaters_in_channel = np.sum(counts_maps) / norm_factor
    total_mass = floaters_in_channel * FLOATER_DENSITY * (FLOATER_RADIUS**3) 
    density = total_mass / hourglass_volume
    return density # kd / nm^3

In [116]:
fg_results = []
for i in ["0", "10", "20", "50", "100", "200"]:
    path = f"../density_test/{i}"
    files = os.listdir(path)
    files_amount = len(files)
    density = 0
    for file in files:
        f_path = f"{path}/{file}"
        density += calculate_fg_density(get_fg_counts_maps(f_path), calc_normalization_factor(f_path))
    density = density/files_amount
    fg_results.append(density)
    print(f"{i} um: {density:.3f} kd/nm^3")

0 um: 0.126 gram/nm^3
10 um: 0.153 gram/nm^3
20 um: 0.149 gram/nm^3
50 um: 0.141 gram/nm^3
100 um: 0.136 gram/nm^3
200 um: 0.117 gram/nm^3


In [122]:
# convert to gram 
KD_TO_GRAM = 1.6605402e-21
NM3_TO_CM3 = 1e-21
for i in fg_results:
    print(f"{(i * KD_TO_GRAM * (1/NM3_TO_CM3)):.3g} g/cm^3")

0.21 g/cm^3
0.254 g/cm^3
0.248 g/cm^3
0.235 g/cm^3
0.226 g/cm^3
0.195 g/cm^3


In [136]:
floater_results = []
for i in ["0", "10", "20", "50", "100", "200"]:
    path = f"../density_test/{i}"
    files = os.listdir(path)
    files_amount = len(files)
    density = 0
    for file in files:
        f_path = f"{path}/{file}"
        density += calculate_floater_density(get_floater_counts_maps(f_path)[0], calc_normalization_factor(f_path))
    density = density/files_amount
    floater_results.append(density)
    print(f"{i} um: \t {density:.3f} kd/nm^3 \t {(density * KD_TO_GRAM * (1/NM3_TO_CM3)):.3g} g/cm^3")

0 um: 	 0.000 kd/nm^3 	 0 g/cm^3
10 um: 	 0.000 kd/nm^3 	 0 g/cm^3
20 um: 	 0.007 kd/nm^3 	 0.0112 g/cm^3
50 um: 	 0.000 kd/nm^3 	 0 g/cm^3
100 um: 	 0.034 kd/nm^3 	 0.0568 g/cm^3
200 um: 	 0.041 kd/nm^3 	 0.068 g/cm^3


In [133]:
for i in floater_results:
    print(f"{(i * KD_TO_GRAM * (1/NM3_TO_CM3)):.3g} g/cm^3")

0 g/cm^3
0 g/cm^3
0.0112 g/cm^3
0 g/cm^3
0.0568 g/cm^3
0.068 g/cm^3
